In [1]:
import os
import datetime
import json
import jsonlines
import difflib
from utils.shared_configs import LLAMA_MODEL_PATH, ZEHPYR_MODEL_PATH, get_sampling_params, initialize_llm

2025-04-17 17:57:35.294566: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 17:57:35.304229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744912655.316180 3503145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744912655.319683 3503145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 17:57:35.332283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
knowledge_base = []
with jsonlines.open('/work/pi_wenlongzhao_umass_edu/8/KB with Summaries/main_dataset.jsonl') as reader:
    for obj in reader:
        knowledge_base.append(obj)

In [3]:
len(knowledge_base), knowledge_base[0]

(204182,
 {'mention': 'Bandar Seri Begawan',
  'cand_name': 'Bandar Seri Begawan',
  'cand_summary': 'Bandar Seri Begawan is the capital city of Brunei with an estimated population of 100,700 and a metropolitan area population of 279,924, governed as a municipality.',
  'cand_text': 'Bandar Seri Begawan (Jawi:<big> بندر سري بڬاوان </big>; ) (formerly known as Brunei Town) is the capital city of the Sultanate of Brunei. It is officially governed as a municipality (). Bandar Seri Begawan has an estimated population of 100,700, and including the whole Brunei-Muara District, the metro area has an estimated population of 279,924.',
  'id': 0,
  'cand_wiki_id': '56611'})

In [4]:
# Build an alias dictionary from the knowledge base that maps lowercased candidate mentions
alias_kb = {}
for entry in knowledge_base:
    key = entry['mention'].lower()
    alias_kb.setdefault(key, []).append(entry)

In [5]:
len(alias_kb)

11585

In [6]:
wn_salience_train = "/project/pi_wenlongzhao_umass_edu/8/data/WN-Salience-2/SED_train.json"
with open(wn_salience_train) as f:
    sed_outputs = json.load(f)

In [7]:
sed_outputs[0]

{'text': "The United States capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning at 6 A.M. local time (1100 UTC), couples began submitting marriage applications at local courthouses citywide.Washington D.C. becomes the seventh United States territory to legalize same sex marriage. The bill was ratified by Mayor Adrian Fenty last December. Due to city's territorial status as a federal district, the bill had to be reviewed by congress. The bill passed congressional review Tuesday night.The bill faced opposition from many family values activists, who tried to stop the bill from becoming law. Supreme Court Chief Justice John Roberts rejected a lawsuit to prevent the measure.",
 'entities': [{'entity_title': 'United States',
   'entity_salience': '1',
   'start': 4,
   'end': 17,
   'left_context': 'The',
   'right_context': 'capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning',
   'entity wiki_ID': '3434750'},
  {'entity_title': 'same-

In [8]:
ent_len=0
for sed_output in sed_outputs:
    for entity in sed_output['entities']:
        ent_len+=1

ent_len

27260

In [19]:
# example to see candidate retrieval from knowledge base
mention = 'congress'

In [9]:
sed_outputs[0]['entities']

[{'entity_title': 'United States',
  'entity_salience': '1',
  'start': 4,
  'end': 17,
  'left_context': 'The',
  'right_context': 'capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning',
  'entity wiki_ID': '3434750'},
 {'entity_title': 'same-sex marriage',
  'entity_salience': '1',
  'start': 56,
  'end': 73,
  'left_context': 'The United States capital of Washington, D.C. legalized',
  'right_context': 'on Wednesday. Beginning at 6 A.M. local time (1100 UTC),',
  'entity wiki_ID': '92656'},
 {'entity_title': 'congress',
  'entity_salience': '1',
  'start': 445,
  'end': 453,
  'left_context': 'a federal district, the bill had to be reviewed by',
  'right_context': 'The bill passed congressional review Tuesday night.The bill faced opposition',
  'entity wiki_ID': ''}]

In [10]:
candidate_freq = {}
for alias, candidates in alias_kb.items():
    for cand in candidates:
        cand_id = cand.get('cand_wiki_id')
        candidate_freq[cand_id] = candidate_freq.get(cand_id, 0) + 1

In [11]:
max_freq = max(candidate_freq.values())
min_freq = min(candidate_freq.values())

In [12]:
len(candidate_freq), max_freq, min_freq

(69513, 628, 1)

In [13]:
def retrieve_candidates(mention_text, alias_dict, cutoff=0.8, alpha=0.5, num_candidates=10):
    mention_lower = mention_text.lower()
    all_keys = list(alias_dict.keys())
    close_keys = difflib.get_close_matches(mention_lower, all_keys, n=len(all_keys), cutoff=cutoff)
    # print(len(close_keys))
    # print(close_keys)
    
    candidates = []
    for key in close_keys:
        candidates.extend(alias_dict[key])
    print('length of candidates before removing duplicates: ', len(candidates))
    
    # Remove duplicates
    seen = set()
    unique_candidates = []
    for cand in candidates:
        # print('cand: ', cand)
        cand_id = cand.get('cand_wiki_id')
        if cand_id not in seen:
            unique_candidates.append(cand)
            seen.add(cand_id)
            
    print('length of candidates after removing duplicates: ', len(unique_candidates))

    if len(unique_candidates)>num_candidates:
        for candidate in unique_candidates:
            candidate_str = candidate.get('mention', '').lower()
            fuzzy_score = difflib.SequenceMatcher(None, mention_lower, candidate_str).ratio()
            # print('fuzzy_score: ', fuzzy_score)
            freq = candidate_freq.get(candidate.get('cand_wiki_id'), 0)
            if max_freq > min_freq:
                norm_freq = (freq - min_freq) / (max_freq - min_freq)
            else:
                norm_freq = 1
            # print('freq: ', freq)
            # candidate['weighted_score'] = fuzzy_score * (freq+1)
            candidate['weighted_score'] = (alpha*fuzzy_score) + ((1 - alpha)*norm_freq)
            # print('weighted score: ', candidate['weighted_score'])
        
        unique_candidates.sort(key=lambda cand: cand.get('weighted_score', 0), reverse=True)
    
    unique_candidates = unique_candidates[:num_candidates]
    
    return unique_candidates

In [20]:
candidates = retrieve_candidates(mention, alias_kb)
len(candidates)

length of candidates before removing duplicates:  520
length of candidates after removing duplicates:  90


10

In [21]:
candidates[0]

{'mention': 'Congress',
 'cand_name': 'United States Congress',
 'cand_summary': 'The United States Congress is a bicameral legislature consisting of the House of Representatives and the Senate, with 535 voting members, including 435 representatives and 100 senators, who are chosen through direct election and serve two-year and six-year terms, respectively, with each state having at least one representative and two senators, and eligibility requirements of age, citizenship, and state residency.',
 'cand_text': 'The United States Congress is the bicameral legislature of the federal government of the United States, and consists of two chambers: the House of Representatives and the Senate. The Congress meets in the United States Capitol in Washington, D.C. Both senators and representatives are chosen through direct election, though vacancies in the Senate may be filled by a gubernatorial appointment. Congress has 535 voting members: 435 representatives and 100 senators. The House of Repre

In [14]:
# def point_wise_el_prompt(entity, candidate):
#     content = "You're an entity disambiguator. I'll give you the description of entity disambiguation and some tips on entity disambiguation, and you need to pay attention to these textual features:\n\n"
#     content += "Context: Look at the surrounding text to understand the topic.\nCategories: Consider the type of the entity (person, organization, location, etc.).\nModifiers: Pay attention to words or phrases that add details to the mention.\nCo-references: Check other mentions of the same entity in the text.\nTemporal and Geographical Factors: Consider when and where the text was written.\nExternal Knowledge: Use outside knowledge not contained in the text.\nRemember, entity disambiguation requires understanding the text, knowing about the world, and using good judgment."
#     content += '\n\n'
#     content += "Now, I'll give you a mention, a context, and a candidate entity, and the mention will be highlighted with '###'.\n\n"
#     content += 'mention:{}\n'.format(entity['entity_title'])

#     context = entity['left_context'] + ' ###' + entity['entity_title'] + '### ' + entity['right_context']
#     context = context.strip()
#     context = ' '.join(context.split())
#     content += 'context:{}\n'.format(context)

#     cand_entity = '{}.{}'.format(candidate['cand_name'], candidate['cand_summary'])
#     content += 'candidate entity:{}\n\n'.format(cand_entity)

#     content += """You need to determine if the mention and the candidate entity are related. Please refer to the above tips and give your reasons, and finally answer 'yes' or 'no'. Answer 'yes' when you think the information is insufficient or uncertain."""

#     return content

def point_wise_el_prompt(entity, candidate):
    prompt = f"""
        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
        You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n
        1. Context: Look at the surrounding text to understand the topic.\n
        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n
        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n
        4. Co-references: Check other mentions of the same entity in the text.\n
        5. Temporal and Geographical Factors: Consider when and where the text was written.\n
        6. External Knowledge: Use knowledge from outside the text.\n
        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n 
        <|eot_id|><|start_header_id|>user<|end_header_id|>\n
        Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'.\n
        mention: {entity['entity_title']} context: {entity['left_context']} ###{entity['entity_title']}### {entity['right_context']}\n
        candidate entity: {candidate['cand_name']}.{candidate['cand_summary']}\n
        Please analyze the information: determine if the mention and the candidate entity are related. Your response should follow this exact structure:
        1. Final Decision (at the beginning): On the very first line, output exactly one line in the following format: Final Decision: yes   OR   Final Decision: no\n
        2. Brief Explanation: Immediately following the final decision, provide a short sentence summarizing why that decision was made.\n
        3. Detailed Reasoning: Provide a thorough explanation of your analysis, covering all relevant points.\n
        4. Final Decision (repeat): At the very end of your response, on a new line, repeat the final decision in the same format as the first line.\n\n
        For example, a correct output would be:\n
        Final Decision: no\n
        I determined that the information is not sufficient to establish a relationship because [brief explanation].\n
        [Your detailed reasoning here...]\n
        Final Decision: no\n
        Remember, answer \"yes\" when you believe the available information is insufficient or uncertain.\n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n
        
    """
    return prompt

# Use the guidelines provided above, outline your reasoning, and finally answer with either \"yes\" or \"no\".
# Answer \"yes\" when you believe the available information is insufficient or uncertain.\n <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n

In [15]:
prompt = point_wise_el_prompt(sed_outputs[0]['entities'][2], candidates[0])
print(prompt)


        <|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:

        1. Context: Look at the surrounding text to understand the topic.

        2. Categories: Consider the type of the entity (person, organization, location, etc.).

        3. Modifiers: Pay attention to words or phrases that add details to the mention.

        4. Co-references: Check other mentions of the same entity in the text.

        5. Temporal and Geographical Factors: Consider when and where the text was written.

        6. External Knowledge: Use knowledge from outside the text.

        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.
 
        <|eot_id|><|start_header_id|>user<|end_header_id|>

        Now, I will provide you with a mention, its cont

In [19]:
outputs = llm.generate(prompts=prompt, sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.10s/it, est. speed input: 173.99 toks/s, output: 44.46 toks/s]


In [20]:
print(outputs[0].outputs[0].text)

 Final Decision: no

I determined that the information is not sufficient to establish a relationship because the context and description of the candidate entity do not match.

The mention "congress" in the context is referring to a legislative body, whereas the candidate entity "Gold" is a chemical element. The context mentions a federal district, a bill, and congressional review, which are all related to politics and governance, whereas the candidate entity is a physical substance with properties and uses. The two entities operate in entirely different domains, making it unlikely that the mention is referring to the candidate entity. The surrounding text does not provide any information that would suggest a connection between the two.

Final Decision: no


## Combine prompts

In [57]:
def get_entity_unique_id(article_index, entity):
    return f"{article_index}_{entity['entity_title']}_{entity['start']}_{entity['end']}"

prompt_id = 0
prompt_collection = []

for article_index, sed_output in enumerate(sed_outputs):
    for entity in sed_output["entities"]:
        entity_unique_id = get_entity_unique_id(article_index, entity)
        entity["entity_unique_id"] = entity_unique_id

        candidates = retrieve_candidates(entity["entity_title"], alias_kb)
        entity['candidates'] = candidates
        # print(len(candidates))
        
        for candidate in candidates:
            prompt = point_wise_el_prompt(entity, candidate)
            # combined_prompt.append(prompt)
            prompt_id += 1
            prompt_collection.append({
                "prompt_id": prompt_id,
                "entity_unique_id": entity_unique_id,
                "prompt": prompt,
                "entity_info": entity, # can update this to not store the candidates again.
                "candidate_info": candidate
            })

length of candidates before removing duplicates:  1721
length of candidates after removing duplicates:  98
length of candidates before removing duplicates:  0
length of candidates after removing duplicates:  0
length of candidates before removing duplicates:  520
length of candidates after removing duplicates:  90
length of candidates before removing duplicates:  1838
length of candidates after removing duplicates:  182
length of candidates before removing duplicates:  520
length of candidates after removing duplicates:  90
length of candidates before removing duplicates:  1838
length of candidates after removing duplicates:  182
length of candidates before removing duplicates:  1721
length of candidates after removing duplicates:  98
length of candidates before removing duplicates:  0
length of candidates after removing duplicates:  0
length of candidates before removing duplicates:  878
length of candidates after removing duplicates:  109
length of candidates before removing duplicat

In [58]:
len(prompt_collection)

147315

In [59]:
prompt_collection[0]

{'prompt_id': 1,
 'entity_unique_id': '0_United States_4_17',
 'prompt': '\n        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n\n        1. Context: Look at the surrounding text to understand the topic.\n\n        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n\n        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n\n        4. Co-references: Check other mentions of the same entity in the text.\n\n        5. Temporal and Geographical Factors: Consider when and where the text was written.\n\n        6. External Knowledge: Use knowledge from outside the text.\n\n        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n \n        <|eot_id|><|st

In [60]:
with open('prompts/pointwise_filter_prompt_collection_4_17.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_collection, f, indent=4)

## Run from here

In [25]:
with open('prompts/pointwise_filter_prompt_collection_4_16.json', 'r', encoding='utf-8') as f:
     prompt_collection = json.load(f)

In [26]:
prompt_collection[0]

{'prompt_id': 1,
 'entity_unique_id': '0_United States_4_17',
 'prompt': '\n        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n\n        1. Context: Look at the surrounding text to understand the topic.\n\n        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n\n        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n\n        4. Co-references: Check other mentions of the same entity in the text.\n\n        5. Temporal and Geographical Factors: Consider when and where the text was written.\n\n        6. External Knowledge: Use knowledge from outside the text.\n\n        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n \n        <|eot_id|><|st

In [27]:
len(prompt_collection)

147315

## Run inference

In [16]:
# import torch
# torch.cuda.empty_cache()
# torch.cuda.reset_peak_memory_stats()
llm = initialize_llm(model_path=LLAMA_MODEL_PATH, tokenizer_path=LLAMA_MODEL_PATH)

INFO 04-17 17:58:05 __init__.py:207] Automatically detected platform cuda.
INFO 04-17 17:58:17 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 04-17 17:58:17 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-17 17:58:17 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-17 17:58:17 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', speculative_config=None, tokenizer='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', skip

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-17 17:58:22 model_runner.py:1115] Loading model weights took 14.9888 GB
INFO 04-17 17:58:23 worker.py:267] Memory profiling takes 0.59 seconds
INFO 04-17 17:58:23 worker.py:267] the current vLLM instance can use total_gpu_memory (44.40GiB) x gpu_memory_utilization (0.90) = 39.96GiB
INFO 04-17 17:58:23 worker.py:267] model weights take 14.99GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 23.70GiB.
INFO 04-17 17:58:24 executor_base.py:111] # cuda blocks: 12136, # CPU blocks: 2048
INFO 04-17 17:58:24 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 1.48x
INFO 04-17 17:58:25 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.89it/s]

INFO 04-17 17:58:43 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.26 GiB
INFO 04-17 17:58:43 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 20.82 seconds


In [17]:
sampling_params = get_sampling_params(max_tokens=200, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

In [18]:
def pointwise_candidate_filtering(llm, sampling_params, article_title, article_text):
    outputs = llm.generate(prompts=combined_prompt, sampling_params=sampling_params)
    return outputs[0].outputs[0].text

In [22]:
def process_batches(prompt_collection, llm, sampling_params, batch_size=1000, output_dir="batch_outputs"):
    all_results = []
    
    prompts = [item["prompt"] for item in prompt_collection]
    # print('prompts: \n', prompts)
    
    for batch_number, start in enumerate(range(0, len(prompts), batch_size), 1):
        batch_prompts = prompts[start : start + batch_size]
        # print('batch_prompts: \n', batch_prompts)
        
        outputs = llm.generate(prompts=batch_prompts, sampling_params=sampling_params)
        batch_timestamp = datetime.datetime.now().isoformat()
        
        batch_results = []
        for idx, output in enumerate(outputs):
            generated_text = output.outputs[0].text
            
            original_item = prompt_collection[start + idx]
            
            result_dict = {
                "iteration": start + idx + 1,
                "batch_number": batch_number,
                "prompt_id": original_item.get("prompt_id"),
                "entity_unique_id": original_item.get("entity_unique_id"),
                "entity": original_item.get("entity_info")["entity_title"],
                "candidate": original_item.get("candidate_info"),
                "llm_pointwise_response": generated_text,
                'prompt': original_item.get("prompt"),
                "timestamp": batch_timestamp
            }
            batch_results.append(result_dict)
            
        all_results.extend(batch_results)
        
        with open(f"outputs/pointwise/batch_{batch_number}_{batch_timestamp.replace(':', '-')}.json", 'w', encoding='utf-8') as f:
            json.dump(batch_results, f, indent=4)
        
        print(f"Saved batch {batch_number} with {len(batch_results)} results.")
    
    return all_results

In [ ]:
batch_size = 7500
all_outputs = process_batches(prompt_collection, llm, sampling_params, batch_size=batch_size)

In [29]:
sed_outputs[0]

{'text': "The United States capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning at 6 A.M. local time (1100 UTC), couples began submitting marriage applications at local courthouses citywide.Washington D.C. becomes the seventh United States territory to legalize same sex marriage. The bill was ratified by Mayor Adrian Fenty last December. Due to city's territorial status as a federal district, the bill had to be reviewed by congress. The bill passed congressional review Tuesday night.The bill faced opposition from many family values activists, who tried to stop the bill from becoming law. Supreme Court Chief Justice John Roberts rejected a lawsuit to prevent the measure.",
 'entities': [{'entity_title': 'United States',
   'entity_salience': '1',
   'start': 4,
   'end': 17,
   'left_context': 'The',
   'right_context': 'capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning',
   'entity wiki_ID': '3434750'},
  {'entity_title': 'same-

In [ ]:
timestamp = datetime.datetime.now().isoformat().replace(":", "-")
with open(f"outputs/pointwise/all_batch_results_{timestamp}.json", "w", encoding="utf-8") as f:
    json.dump(all_outputs, f, indent=4)

In [ ]:
batch_dir = "outputs/pointwise"
batch_pattern = re.compile(r"batch_(\d+)_.*\.json")

batch_files = [
    f for f in os.listdir(batch_dir)
    if batch_pattern.match(f) and f.endswith(".json")
]

batch_files_sorted = sorted(
    batch_files,
    key=lambda f: int(batch_pattern.match(f).group(1))
)

all_batches = []

for file_name in batch_files_sorted:
    file_path = os.path.join(batch_dir, file_name)
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            all_batches.append(data)
    except json.JSONDecodeError:
        print(f"Warning: Failed to decode {file_name}, skipping.")
    except Exception as e:
        print(f"Error with {file_name}: {e}")

print(len(all_batches))
output_path = os.path.join(batch_dir, "combined_batch_output.json")
with open(output_path, 'w') as out_file:
    json.dump(all_batches, out_file, indent=2)

print(f"Combined output saved to: {output_path}")

In [18]:
with open(f"outputs/pointwise/pointwise_iteration_1/combined_batch_output.json", "r", encoding="utf-8") as f:
    pointwise_combined_outputs = json.load(f)

In [19]:
len(pointwise_combined_outputs),len(pointwise_combined_outputs[0])

(148, 2000)

In [20]:
pointwise_combined_outputs[0][0]

{'iteration': 1,
 'batch_number': 1,
 'prompt_id': 1,
 'entity': 'United States',
 'candidate': {'mention': 'United States',
  'cand_name': 'United States',
  'cand_summary': "The United States of America is a federal republic and representative democracy, comprising 50 states, five major self-governing territories, and various possessions, with an estimated population of over 327 million people, the third most populous country in the world, and a highly developed country with the world's largest economy by nominal GDP, second-largest by purchasing power parity, and a leading political, cultural, and scientific force internationally.",
  'cand_text': "The United States of America (USA), commonly known as the United States (U.S. or US) or simply America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8&nbsp;million square miles (9.8&nbsp;million km<sup>2</sup>), it is the world's third or fourth largest country

In [26]:
len(pointwise_combined_outputs)

148

In [28]:
negative_keywords = ['"no"', "'no'", "not related", "cannot establish a relationship"]

final_output = []
for pointwise_combined_output in pointwise_combined_outputs:
    for output in pointwise_combined_output:
        if any(neg in output['generated_text'] for neg in negative_keywords):
            continue
        else:
            final_output.append(output)
            
len(final_output)

49079

In [32]:
with open(f"outputs/pointwise/pointwise_iteration_1/removing_neg.json", 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4)

In [21]:
def extract_final_decision(llm_response):
    """
    Expects at least one line starting with "Final Decision:".]
    """
    lines = llm_response.strip().splitlines()
    for line in reversed(lines):
        if line.strip().startswith("Final Decision:"):
            decision = line.split("Final Decision:")[1].strip().lower()
            return decision
    return None

In [100]:
negative_keywords = ['"no"', "'no'", "not related", "cannot establish a relationship"]

results_by_entity = {}

for result in all_outputs:
    response_text = result.get("llm_pointwise_response", "")
    decision = extract_final_decision(response_text)
    # if information is insufficient, assume yes
    if decision is None:
        decision = "yes"
    
    if any(neg in decision for neg in negative_keywords):
        continue
    else:
        entity_uid = result["entity_unique_id"]
        if entity_uid not in results_by_entity:
            results_by_entity[entity_uid] = {
                "entity": result["entity"],
                "filtered_candidates": []
            }
        results_by_entity[entity_uid]["filtered_candidates"].append(result["candidate"])

# update each entity with filtered candidate list
for article_index, sed_output in enumerate(sed_outputs):
    for entity in sed_output["entities"]:
        uid = entity.get("entity_unique_id")
        if uid in results_by_entity:
            entity["filtered_candidates"] = results_by_entity[uid]["filtered_candidates"]
        else:
            entity["filtered_candidates"] = []

with open("outputs/pointwise/pointwise_filtering_final_output.json", "w", encoding="utf-8") as fout:
    json.dump(sed_outputs, fout, indent=4)

{'entity_title': 'United States', 'entity_salience': '1', 'start': 4, 'end': 17, 'left_context': 'The', 'right_context': 'capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning', 'entity wiki_ID': '3434750', 'candidates': {'mention': 'New York', 'cand_name': 'September 11 attacks', 'cand_summary': 'On September 11, 2001, a series of coordinated terrorist attacks were carried out by al-Qaeda against the United States, resulting in the deaths of 2,977 people, including 19 hijackers, and causing extensive damage to the World Trade Center in New York City and the Pentagon in Washington D.C., with a fourth plane crashing in a field in Pennsylvania.', 'cand_text': "The September 11 attacks (also referred to as 9/11) were a series of four coordinated terrorist attacks by the Islamic terrorist group al-Qaeda against the United States on the morning of Tuesday, September 11, 2001. The attacks resulted in 2,977 victim fatalities, over 25,000 injuries, and caused at least